In [1]:
# necessary import statements
from transformers import AutoTokenizer, TFAutoModelForCausalLM
import pandas as pd
import os

c:\Users\alecc\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load trained transformer model
tf_model = TFAutoModelForCausalLM.from_pretrained('../trained_models/gpt2-summarization/')
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Constants
TLDR = ' TL;DR '
MAX_LEN = 512

ResourceExhaustedError: {{function_node __wrapped__AssignVariableOp_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[50257,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:AssignVariableOp]

In [28]:
ah = {} # actual headlines for each dataset
ph = {} # predicted headlines for each dataset

def summarize_article_tf(model, article):
    """ Summarize a given article using the fine-tuned GPT-2 model"""
    tokenized = tokenizer(article, return_tensors="np")
    outputs = model.generate(**tokenized, max_new_tokens=32, pad_token_id=50256)
    return tokenizer.decode(outputs[0])

# Load results_tf if it exists 
if 'results_tf.csv' in os.listdir('../data/'):
    preds_df = pd.read_csv('../data/results_tf.csv')
    preds = {'predicted headline': preds_df['predicted headline'].tolist(), 'actual headline': preds_df['actual headline'].tolist()}
else:
    preds = {'predicted headline': [], 'actual headline': []}

# Generate headlines using the test data
with open('../data/test_data.txt', encoding='utf-8') as f:
    idx = len(preds_df) - 1
    lines = f.readlines()[idx:1000]
    for line in lines:
        idx += 1
        article, actual_headline = line.strip().split(TLDR)
        article = article + TLDR
        ah[idx] = actual_headline
        predicted_headline = summarize_article_tf(tf_model, article).split(TLDR)[1].replace('<|endoftext|>', '.').strip()
        ph[idx] = predicted_headline
        preds['predicted headline'].append(predicted_headline)
        preds['actual headline'].append(actual_headline)
        print(f'{idx}:')
        print(f'\tactual: {actual_headline}')
        print(f'\tpredic: {predicted_headline}')
        if idx % 50 == 0 and idx != 0:
            preds_df = pd.DataFrame.from_dict(preds, orient='columns')
            preds_df.to_csv('../data/results_tf.csv')